In [2]:
import networkx as nx
import pickle
import sys
import os
import glob

# Add the parent directory to sys.path to allow importing from src
# In a Jupyter notebook, we can't use __file__, so we'll use a different approach

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory (src)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
	sys.path.append(parent_dir)

# Now try to import the modules
try:
	from net_filtering import filtering_techniques
	from benchmark.net_filter_bench import bench_net2net_filtering
	from benchmark.net2net_indicators import calculate_jaccard_distance
except ImportError as e:
	print(f"Error importing modules: {e}")
	print("Please ensure that the 'src' directory structure is correct and contains the necessary modules.")
	raise

# Get all simulated network files
simulated_nets_path = '../../data/simulated_nets/*.pickle'
simulated_net_files = glob.glob(simulated_nets_path)

if not simulated_net_files:
	print(f"Error: No pickle files found in {simulated_nets_path}")
	print("Please ensure that the directory exists and contains pickle files.")
	raise FileNotFoundError

# List of filtering techniques
filtering_funcs = [
	filtering_techniques.global_threshold_filter,
	filtering_techniques.local_degree_sparsifier,
	filtering_techniques.random_edge_sparsifier,
	filtering_techniques.simmelian_sparsifier,
	filtering_techniques.disparity_filter
]

# Run bench_net2net_filtering for each technique and each network
results = {}
for net_file in simulated_net_files:
	net_name = os.path.basename(net_file).split('.')[0]
	results[net_name] = {}

	# Load the network
	try:
		with open(net_file, 'rb') as f:
			original_net = pickle.load(f)
	except Exception as e:
		print(f"Error loading {net_file}: {e}")
		continue

	for func in filtering_funcs:
		# Prepare the filter function with default parameters
		if func == filtering_techniques.global_threshold_filter:
			filter_func = lambda G: func(G, attribute='weight', threshold=0.5)
		elif func in [filtering_techniques.local_degree_sparsifier, filtering_techniques.random_edge_sparsifier]:
			filter_func = lambda G: func(G, target_ratio=0.5)
		else:
			filter_func = func

		# Run the benchmark
		score = bench_net2net_filtering(original_net, filter_func, calculate_jaccard_distance)
		results[net_name][func.__name__] = score

# Print results
for net_name, net_results in results.items():
	print(f"\nResults for {net_name}:")
	for technique, score in net_results.items():
		print(f"  {technique}: Jaccard Distance = {score:.4f}")



Results for random_graph:
  global_threshold_filter: Jaccard Distance = 1.0000
  local_degree_sparsifier: Jaccard Distance = 0.5000
  random_edge_sparsifier: Jaccard Distance = 0.5000
  simmelian_sparsifier: Jaccard Distance = 0.2848
  disparity_filter: Jaccard Distance = 1.0000

Results for barabasi_albert_graph:
  global_threshold_filter: Jaccard Distance = 1.0000
  local_degree_sparsifier: Jaccard Distance = 0.5000
  random_edge_sparsifier: Jaccard Distance = 0.5000
  simmelian_sparsifier: Jaccard Distance = 0.0000
  disparity_filter: Jaccard Distance = 1.0000

Results for grid_graph:
  global_threshold_filter: Jaccard Distance = 1.0000
  local_degree_sparsifier: Jaccard Distance = 0.5000
  random_edge_sparsifier: Jaccard Distance = 0.5000
  simmelian_sparsifier: Jaccard Distance = 0.0000
  disparity_filter: Jaccard Distance = 1.0000
